In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from os.path import join
from absl import flags
import os
import sys
import csv
import pandas as pd
import collections
import numpy as np
import time
import math
import json
import random
from copy import copy
from collections import defaultdict as dd

import absl.logging as _logging  # pylint: disable=unused-import
import tensorflow as tf

import sentencepiece as spm

from data_utils import SEP_ID, VOCAB_SIZE, CLS_ID
import model_utils
import function_builder
from classifier_utils import PaddingInputExample
from classifier_utils import convert_single_example, convert_single_example_dev
from prepro_utils import preprocess_text, encode_ids

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
tf.reset_default_graph()
tf.keras.backend.clear_session()

# Model
flags.DEFINE_string("model_config_path", default="/home/calvin/xlnet_cased_L-12_H-768_A-12/xlnet_config.json",
      help="Model config path.")
flags.DEFINE_float("dropout", default=0.1,
      help="Dropout rate.")
flags.DEFINE_float("dropatt", default=0.1,
      help="Attention dropout rate.")
flags.DEFINE_integer("clamp_len", default=-1,
      help="Clamp length")
flags.DEFINE_string("summary_type", default="last",
      help="Method used to summarize a sequence into a compact vector.")
flags.DEFINE_bool("use_summ_proj", default=True,
      help="Whether to use projection for summarizing sequences.")
flags.DEFINE_bool("use_bfloat16", False,
      help="Whether to use bfloat16.")

# Parameter initialization
flags.DEFINE_enum("init", default="normal",
      enum_values=["normal", "uniform"],
      help="Initialization method.")
flags.DEFINE_float("init_std", default=0.02,
      help="Initialization std when init is normal.")
flags.DEFINE_float("init_range", default=0.1,
      help="Initialization std when init is uniform.")

# I/O paths
flags.DEFINE_bool("overwrite_data", default=False,
      help="If False, will use cached data if available.")
flags.DEFINE_string("init_checkpoint", default="/home/calvin/xlnet_cased_L-12_H-768_A-12/xlnet_model.ckpt",
      help="checkpoint path for initializing the model. "
      "Could be a pretrained model or a finetuned model.")
flags.DEFINE_string("output_dir", default="proc_data/sts-b",
      help="Output dir for TF records.")
flags.DEFINE_string("spiece_model_file", default="/home/calvin/xlnet_cased_L-12_H-768_A-12/spiece.model",
      help="Sentence Piece model path.")
flags.DEFINE_string("model_dir", default="exp",
      help="Directory for saving the finetuned model.")
flags.DEFINE_string("data_dir", default="/home/calvin/111/glue_data/STS-B/",
      help="Directory for input data.")

# TPUs and machines
flags.DEFINE_bool("use_tpu", default=False, help="whether to use TPU.")
flags.DEFINE_integer("num_hosts", default=1, help="How many TPU hosts.")
flags.DEFINE_integer("num_core_per_host", default=1,
      help="8 for TPU v2 and v3-8, 16 for larger TPU v3 pod. In the context "
      "of GPU training, it refers to the number of GPUs used.")
flags.DEFINE_string("tpu_job_name", default=None, help="TPU worker job name.")
flags.DEFINE_string("tpu", default=None, help="TPU name.")
flags.DEFINE_string("tpu_zone", default=None, help="TPU zone.")
flags.DEFINE_string("gcp_project", default=None, help="gcp project.")
flags.DEFINE_string("master", default=None, help="master")
flags.DEFINE_integer("iterations", default=400,         ################ training loops
      help="number of iterations per TPU training loop.")

# training
flags.DEFINE_bool("do_train", default=True, help="whether to do training")
flags.DEFINE_integer("train_steps", default=300000,        ################ training loops
      help="Number of training steps")
flags.DEFINE_integer("warmup_steps", default=120, help="number of warmup steps")
flags.DEFINE_float("learning_rate", default=2.5e-6, help="initial learning rate")
# flags.DEFINE_float("learning_rate", default=1e-5, help="initial learning rate") #### original LR
flags.DEFINE_float("lr_layer_decay_rate", 1.0,
                   "Top layer: lr[L] = FLAGS.learning_rate."
                   "Low layer: lr[l-1] = lr[l] * lr_layer_decay_rate.")
flags.DEFINE_float("min_lr_ratio", default=0.0,
      help="min lr ratio for cos decay.")
flags.DEFINE_float("clip", default=1.0, help="Gradient clipping")
flags.DEFINE_integer("max_save", default=0,
      help="Max number of checkpoints to save. Use 0 to save all.")
flags.DEFINE_integer("save_steps", default=6000,  #### how often save check point in xlnet/exp/sts-b
      help="Save the model for every save_steps. "
      "If None, not to save any model.")
flags.DEFINE_integer("star_from_dev", default=11, ## start from which checkpoint in the checkpoint list
      help="start from which checkpoint. "
      "If None, not to save any model.")

flags.DEFINE_integer("train_batch_size", default=6, #train_batch_size  = eval_batch_size,predict_batch_size
      help="Batch size for training")
flags.DEFINE_float("weight_decay", default=0.00, help="Weight decay rate")
flags.DEFINE_float("adam_epsilon", default=1e-8, help="Adam epsilon")
flags.DEFINE_string("decay_method", default="poly", help="poly or cos")

# evaluation
flags.DEFINE_bool("do_eval", default=True, help="whether to do eval")
flags.DEFINE_bool("do_predict", default=False, help="whether to do prediction")
flags.DEFINE_float("predict_threshold", default=0,
      help="Threshold for binary prediction.")
flags.DEFINE_string("eval_split", default="dev", help="could be dev or test")
flags.DEFINE_integer("eval_batch_size", default=6,
      help="batch size for evaluation")
flags.DEFINE_integer("predict_batch_size", default=6,
      help="batch size for prediction.")
flags.DEFINE_string("predict_dir", default='/home/calvin/xlnet/predict_result/',
      help="Dir for saving prediction files.")
flags.DEFINE_bool("eval_all_ckpt", default=True,
      help="Eval all ckpts. If False, only evaluate the last one.")
flags.DEFINE_string("predict_ckpt", default=None,
      help="Ckpt path for do_predict. If None, use the last one.")

# task specific
flags.DEFINE_string("task_name", default="sts-b", help="Task name")
flags.DEFINE_integer("max_seq_length", default=1024, help="Max sequence length")
# flags.DEFINE_integer("max_seq_length", default=1024, help="Max sequence length")
flags.DEFINE_integer("shuffle_buffer", default=2048,
      help="Buffer size used for shuffle.")
flags.DEFINE_integer("num_passes", default=1,
      help="Num passes for processing training data. "
      "This is use to batch data without loss for TPUs.")
flags.DEFINE_bool("uncased", default=False,
      help="Use uncased.")
flags.DEFINE_string("cls_scope", default=None,
      help="Classifier layer scope.")
flags.DEFINE_bool("is_regression", default=True,
      help="Whether it's a regression task.")

FLAGS = flags.FLAGS



In [ ]:


# In[ ]:


from tensorflow.python.keras.preprocessing.text import text_to_word_sequence
import os
import numpy as np
import random
from numpy.random import choice
np.random.seed(1)


# In[ ]:


os.environ['HDF5_USE_FILE_LOCKING'] = 'FALSE'

class ScoreReader():
    def __init__(self, csv_path, cv_directory,
                 system_idx=-1, fname_idx=0, score_idx=2, testname_idx=None,
                 count_words=True, nan_val=''
                ):
        self.csv_path = csv_path
        self.cv_directory = cv_directory  # in order to prune non-existing files
        self.system_idx = system_idx
        self.fname_idx = fname_idx
        self.score_idx = score_idx
        self.testname_idx = testname_idx
        self.testnames = []
        self.nan_val = nan_val
        self.read_scores(count_words)
        random.shuffle(self.scores)
        if not isinstance(self.score_idx, list):
            s = pd.Series([t[1] for t in self.scores])
            s[s<2500].plot.hist(bins=10, alpha=0.5)

    def get_word_len(self, filename):
        with open(self.get_file_path(filename), 'r') as fh:
            lines = fh.read()
            return len(text_to_word_sequence(lines))
        return None

    def get_file_path(self, filename):
        return self.cv_directory + filename

    @classmethod
    def percentile2zscore(cls, percentile):
        if percentile < 1:
            percentile = 1
        elif percentile > 99:
            percentile = 99
        return 0.5 + 0.4 * ss.norm.ppf(percentile / 100)

    @classmethod
    def sten2zscore(cls, sten):
        #Sten scores (for the entire population of results) have a mean of 5.5 and a standard deviation of 2.[1]
        # z-scores have a mean of 0 and a standard deviation of 1
        return (sten - 5.5) / 2

    def get_score(self, fields, score_idx, testname_idx=None):
        if isinstance(score_idx, list):
            scores = []
            for score_ix1 in score_idx:
                scores.append(self.get_score(fields, score_ix1, testname_idx))
            return scores
        else:
            val = fields[score_idx]
            if val != self.nan_val:
                score = float(val)
            else:
                score = np.nan
            if testname_idx is not None:
                self.testnames.append(fields[testname_idx].lower())
                # normalize:
                if 'sten' in fields[testname_idx].lower():
                    score = score  # *= 10
                elif 'percentile' in fields[testname_idx].lower():
                    score /= 10.1
                elif 'zero' in fields[testname_idx].lower():
                    if score == 0.0:
                        score = None
                    else:
                        score /= 10.1
            if score is not None:
                return score
            else:
                return None

    def get_scores(self):
        return np.array([score for _, score in self.scores])

    def set_scores(self, scores):
        if isinstance(scores, np.ndarray):
            print('converting scores to list of lists...')
            scores = scores.tolist()
        assert len(scores) == len(self.scores)
        filenames = [filename for filename, _ in self.scores]
        self.scores = [(filename, score) for filename, score in zip(filenames, scores)]

    def normalize_scores(self, ranking=False):
        raw_scores = self.get_scores()
        if raw_scores.ndim < 2:
            raw_scores = raw_scores.reshape(-1, 1)
        if ranking:
            scores = ss.rankdata(raw_scores)
            self.set_scores(raw_scores)
        else:
            new_scores = (
                raw_scores - np.nanmean(raw_scores, axis=0)
            ) / np.nanstd(raw_scores, axis=0)
            new_scores = [
                val if len(val)>1 else val[0]
                for val in new_scores.tolist()
            ]
            self.set_scores(new_scores)

    def read_scores(self, count_words=True):
        # read scores from csv file
        with open(self.csv_path, 'r') as fh:
            lines = fh.readlines()
        
        print('parsing {} scores...'.format(len(lines)-1))
        self.scores = []
        self.word_count = []
        skip = True
        for line in lines:
            if skip:
                print(line)
                fields = line.split(',')
                def print_labels(field_idx, name):
                    if field_idx is not None:
                        if isinstance(field_idx, list):
                            print('{}: {}'.format(name, [fields[idx] for idx in field_idx]))
                        else:
                            print('{}: {}'.format(name, fields[field_idx]))
                print_labels(self.system_idx, 'system index')
                print_labels(self.score_idx, 'score index')
                print_labels(self.testname_idx, 'testname')
                print_labels(self.fname_idx, 'filename_idx')
                skip = False
            else:
                fields = line.split(',')
                fields[-1] = fields[-1].strip()
                filename = str(fields[self.system_idx]).zfill(3) + '/' + fields[self.fname_idx] + '.txt'
                if os.path.exists(self.get_file_path(filename)):
                    score = self.get_score(fields, self.score_idx, self.testname_idx)
                    
                    
                    if score is not None:
############################################################# delete data
                        if self.get_word_len(filename) > 100:
                            self.scores.append((filename, score))
############################################################# delete data  
                    if count_words:
                        self.word_count.append(self.get_word_len(filename))
                else:
                    print('{} not found'.format(filename))
                
        if len(self.scores) == 0:
            raise ValueError('No data parsed!')
        if not isinstance(self.score_idx, list):
            score_vocab = set([score for _, score in self.scores])
            print('{} different score values found'.format(len(score_vocab)))
            print(score_vocab)
        else:
            print('dimensionality: {}'.format(len(self.scores[0][1])))
        print('{} scores parsed'.format(len(self.scores)))

    def split_train_test(self, train_fraction=0.9, val_fraction=0.6):
        split = train_fraction * len(self.scores)
        
        trainScores = []
        registerScores = []
        for count, score in enumerate(self.scores):
            if count<split:
                trainScores.append(score)
            else:
                registerScores.append(score)
        
        valScores = []
        testScores = []
        split1 = val_fraction * len(registerScores)
        for count, score in enumerate(registerScores):
            if count<split1:
                valScores.append(score)
            else:
                testScores.append(score)
                
        print('{} train scores, {} val scores, {} test scores'.format(len(trainScores), len(valScores),len(testScores)))
        return trainScores, valScores, testScores


# In[ ]:


class InputExample(object):
  """A single training/test example for simple sequence classification."""

  def __init__(self, guid, text_a, text_b=None, label=None):
    """Constructs a InputExample.
    Args:
      guid: Unique id for the example.
      text_a: string. The untokenized text of the first sequence. For single
        sequence tasks, only this sequence must be specified.
      text_b: (Optional) string. The untokenized text of the second sequence.
        Only must be specified for sequence pair tasks.
      label: (Optional) string. The label of the example. This should be
        specified for train and dev examples, but not for test examples.
    """
    self.guid = guid
    self.text_a = text_a
    self.text_b = text_b
    self.label = label


class DataProcessor(object):
  """Base class for data converters for sequence classification data sets."""

  def get_labels(self):
    """Gets the list of labels for this data set."""
    raise NotImplementedError()


class GLUEProcessor(DataProcessor):
    def __init__(self, filename,label,filename_dev,label_dev,filename_test,label_test):
        self.contains_header = True
        self.test_text_a_column = None
        self.test_text_b_column = None
        self.test_contains_header = True
        self.cv_directory='/u02/texts/'
        self.filename=filename           # all filename (list)
        self.label=label                 # all label (list)
        self.filename_dev=filename_dev   # all filename (list)
        self.label_dev=label_dev         # all label (list)
        self.filename_test=filename_test # all filename (list)
        self.label_test=label_test       # all label (list)
        
#######################################################################################################
    def get_file_path(self, filename):
        return self.cv_directory + filename

    def __get_text__(self, filename,set_type,filename_dev,filename_test):
        if set_type == "train":
            lines=[]
            for i,data in enumerate(filename):
                with open(self.get_file_path(data), 'r', encoding='utf-8') as fh:
                    line = fh.read()
                    lines.append([line])
            return lines
        
        if set_type == "dev":
            lines=[]
            for i,data in enumerate(filename_dev):
                with open(self.get_file_path(data), 'r', encoding='utf-8') as fh:
                    line = fh.read()
                    lines.append([line])
            return lines     
        
        if set_type == "test":
            lines=[]
            for i,data in enumerate(filename_test):
                with open(self.get_file_path(data), 'r', encoding='utf-8') as fh:
                    line = fh.read()
                    lines.append([line])
            return lines
#######################################################################################################

    def get_train_examples(self, data_dir, filename,filename_dev,filename_test, 
                           label,label_dev,label_test):
        """See base class."""
        return self._create_examples(
            self.__get_text__(filename, "train",filename_dev,filename_test),
            "train", label,label_dev,label_test)

    def get_dev_examples(self, data_dir, filename,filename_dev,filename_test, 
                         label,label_dev,label_test):
        """See base class."""
        return self._create_examples(
            self.__get_text__(filename,"dev",filename_dev,filename_test),
            "dev", label,label_dev,label_test)

    def get_test_examples(self, data_dir, filename,filename_dev,filename_test,
                          label,label_dev,label_test):
        """See base class."""
        return self._create_examples(
            self.__get_text__(filename,"test",filename_dev,filename_test),
            "test", label,label_dev,label_test)

    def get_labels(self, num=31):
#         return [0.0]
        return [str(i) for i in range(num)]

    # lines: all raw data (list), set_type: train or dev or testing
    # create all examples
    def _create_examples(self, lines, set_type, label, label_dev,label_test):
        """Creates examples for the training and dev sets."""
        if set_type == "train":
            examples = []
            for i, line in enumerate(lines):
                guid = "%s-%s" % (set_type, i)
                line=line
                label_a=label[i]  
                examples.append(
                  InputExample(guid=guid, text_a=line, label=label_a))
            return examples
        
        if set_type == "dev":
            examples = []
            for j, line in enumerate(lines):
                guid = "%s-%s" % (set_type, j)
                line=line
                label_b=label_dev[j]  
                examples.append(
                  InputExample(guid=guid, text_a=line, label=label_b))
            return examples        

        if set_type == "test":
            examples = []
            for k, line in enumerate(lines):
                guid = "%s-%s" % (set_type, k)
                line=line
                label_c=label_test[k]  
                examples.append(
                  InputExample(guid=guid, text_a=line, label=label_c))
            return examples   
        
# examples: only one dataset
#guid:train-1, train-2 ....
# text_a: raw data
# label: score 


# In[ ]:


def file_based_convert_examples_to_features(
    examples, label_list, max_seq_length, tokenize_fn, output_file,
    num_passes=1):
  """Convert a set of `InputExample`s to a TFRecord file."""

  # do not create duplicated records
  if tf.gfile.Exists(output_file) and not FLAGS.overwrite_data:
    tf.logging.info("Do not overwrite tfrecord {} exists.".format(output_file))
    return

  tf.logging.info("Create new tfrecord {}.".format(output_file))

  writer = tf.python_io.TFRecordWriter(output_file)

  if num_passes > 1:
    examples *= num_passes


  for (ex_index, example) in enumerate(examples):                 # iterating the data#############
    if ex_index % 10000 == 0:
      tf.logging.info("Writing example {} of {}".format(ex_index,
                                                        len(examples)))
### feaure: one data
# input_ids=input_ids,
# input_mask=input_mask,
# segment_ids=segment_ids,
# label_id=label_id)
    feature = convert_single_example(ex_index, example, label_list,
                                     max_seq_length, tokenize_fn)

    def create_int_feature(values):
      f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
      return f

    def create_float_feature(values):
      f = tf.train.Feature(float_list=tf.train.FloatList(value=list(values)))
      return f

    features = collections.OrderedDict()
    features["input_ids"] = create_int_feature(feature.input_ids)
    features["input_mask"] = create_float_feature(feature.input_mask)
    features["segment_ids"] = create_int_feature(feature.segment_ids)
    

    if label_list is not None:        ###go into this requirement###
      features["label_ids"] = create_float_feature([float(i) for i in feature.label_id]) ## error ##
    else:
      features["label_ids"] = create_float_feature([float(feature.label_id)])
    features["is_real_example"] = create_int_feature(
        [int(feature.is_real_example)])



    tf_example = tf.train.Example(features=tf.train.Features(feature=features))
    writer.write(tf_example.SerializeToString())
  writer.close()


def file_based_input_fn_builder(input_file, seq_length, is_training,
                                drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""


  name_to_features = {
      "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "input_mask": tf.FixedLenFeature([seq_length], tf.float32),
      "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "label_ids": tf.FixedLenFeature([31], tf.float32),
#       "label_ids": tf.FixedLenFeature([], tf.int64),
      "is_real_example": tf.FixedLenFeature([], tf.int64),
  }
  if FLAGS.is_regression:
    name_to_features["label_ids"] = tf.FixedLenFeature([31], tf.float32)
#     name_to_features["label_ids"] = tf.FixedLenFeature([], tf.float32)
  tf.logging.info("Input tfrecord file {}".format(input_file))

  def _decode_record(record, name_to_features):
    """Decodes a record to a TensorFlow example."""
    example = tf.parse_single_example(record, name_to_features)

    # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
    # So cast all int64 to int32.
    for name in list(example.keys()):
      t = example[name]
      if t.dtype == tf.int64:
        t = tf.cast(t, tf.int32)
      example[name] = t

    return example

  def input_fn(params, input_context=None):
    """The actual input function."""
    if FLAGS.use_tpu:
      batch_size = params["batch_size"]
    elif is_training:
      batch_size = FLAGS.train_batch_size
    elif FLAGS.do_eval:
      batch_size = FLAGS.eval_batch_size
    else:
      batch_size = FLAGS.predict_batch_size

    d = tf.data.TFRecordDataset(input_file)
    # Shard the dataset to difference devices
    if input_context is not None:
      tf.logging.info("Input pipeline id %d out of %d",
          input_context.input_pipeline_id, input_context.num_replicas_in_sync)
      d = d.shard(input_context.num_input_pipelines,
                  input_context.input_pipeline_id)

    # For training, we want a lot of parallel reading and shuffling.
    # For eval, we want no shuffling and parallel reading doesn't matter.
    if is_training:
      d = d.shuffle(buffer_size=FLAGS.shuffle_buffer)
      d = d.repeat()

    d = d.apply(
        tf.contrib.data.map_and_batch(
            lambda record: _decode_record(record, name_to_features),
            batch_size=batch_size,
            drop_remainder=drop_remainder))

    return d

  return input_fn


# model_fn = get_model_fn(len(label_list) if label_list is not None else None)
# n_class no need to input
def get_model_fn(n_class):
    def model_fn(features, labels, mode, params):
    #### Training or Evaluation 
        
        is_training = (mode == tf.estimator.ModeKeys.TRAIN)
        #### Get loss from inputs
        if FLAGS.is_regression:
            (total_loss, per_example_loss, logits
              ) = function_builder.get_regression_loss(FLAGS, features, n_class, is_training, mode)
        else:
            (total_loss, per_example_loss, logits,FLAGS, features, n_class, is_training)

        #### Check model parameters
        num_params = sum([np.prod(v.shape) for v in tf.trainable_variables()])
        tf.logging.info('#params: {}'.format(num_params))

        #### load pretrained models
        scaffold_fn = model_utils.init_from_checkpoint(FLAGS)

        #### Evaluation mode
        if mode == tf.estimator.ModeKeys.EVAL:
            assert FLAGS.num_hosts == 1

        
############################## criteria #################################### didn't be used 
            def metric_fn(per_example_loss, label_ids, logits, is_real_example):
                predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
                eval_input_dict = {
                    'labels': label_ids,
                    'predictions': predictions,
                    'weights': is_real_example
                }
                accuracy = tf.metrics.accuracy(**eval_input_dict)

                loss = tf.metrics.mean(values=per_example_loss, weights=is_real_example)
                return {
                    'eval_accuracy': accuracy,
                    'eval_loss': loss}
############################## criteria ####################################


########### my new version 
############################## criteria ####################################
            def regression_metric_fn(  
                per_example_loss, label_ids, logits):

                loss = tf.metrics.mean(values=per_example_loss)

                return {'eval_loss': loss}
############################## criteria ####################################

            is_real_example = tf.cast(features["is_real_example"], dtype=tf.float32)

            #### Constucting evaluation TPUEstimatorSpec with new cache.
            label_ids = tf.reshape(features['label_ids'], [-1])

            if FLAGS.is_regression:
                metric_fn = regression_metric_fn
            else:
                metric_fn = metric_fn
            metric_args = [per_example_loss, label_ids, logits]

            if FLAGS.use_tpu:
                eval_spec = tf.contrib.tpu.TPUEstimatorSpec(
                    mode=mode,
                    loss=total_loss,
                    eval_metrics=(metric_fn, metric_args),
                    scaffold_fn=scaffold_fn)
            else:
                eval_spec = tf.estimator.EstimatorSpec(
                    mode=mode,
                    loss=total_loss,
                    eval_metric_ops=metric_fn(*metric_args))

            return eval_spec

#### Prediction ###########################################################
        if mode == tf.estimator.ModeKeys.PREDICT:
            label_ids = tf.reshape(features["label_ids"], [-1])

            predictions = {
              "logits": logits,
              "labels": label_ids,
              "is_real": features["is_real_example"]
            }

            if FLAGS.use_tpu:
                output_spec = tf.contrib.tpu.TPUEstimatorSpec(
                    mode=mode, predictions=predictions, scaffold_fn=scaffold_fn)
            else:
                output_spec = tf.estimator.EstimatorSpec(
                    mode=mode, predictions=predictions)
            return output_spec

    #### Configuring the optimizer

        train_op, learning_rate, _ = model_utils.get_train_op(FLAGS, total_loss)

        monitor_dict = {}
        monitor_dict["lr"] = learning_rate

    #### Constucting training TPUEstimatorSpec with new cache.
        if FLAGS.use_tpu:
      #### Creating host calls
            if not FLAGS.is_regression:
                label_ids = tf.reshape(features['label_ids'], [-1])
                predictions = tf.argmax(logits, axis=-1, output_type=label_ids.dtype)
                is_correct = tf.equal(predictions, label_ids)
                accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

                monitor_dict["accuracy"] = accuracy

                host_call = function_builder.construct_scalar_host_call(
                    monitor_dict=monitor_dict,
                    model_dir=FLAGS.model_dir,
                    prefix="train/",
                    reduce_fn=tf.reduce_mean)
            else:
                host_call = None

            train_spec = tf.contrib.tpu.TPUEstimatorSpec(
                mode=mode, loss=total_loss, train_op=train_op, host_call=host_call,
                scaffold_fn=scaffold_fn)
        else:
            train_spec = tf.estimator.EstimatorSpec(
                mode=mode, loss=total_loss, train_op=train_op)

        return train_spec

    return model_fn


def file_based_input_fn_builder_dev(input_file, seq_length, is_training,
                                drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""


  name_to_features = {
      "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "input_mask": tf.FixedLenFeature([seq_length], tf.float32),
      "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "label_ids": tf.FixedLenFeature([31], tf.float32),
#       "label_ids": tf.FixedLenFeature([], tf.int64),
      "is_real_example": tf.FixedLenFeature([], tf.int64),
  }
  if FLAGS.is_regression:
    name_to_features["label_ids"] = tf.FixedLenFeature([31], tf.float32)
#     name_to_features["label_ids"] = tf.FixedLenFeature([], tf.float32)
  tf.logging.info("Input tfrecord file {}".format(input_file))

  def _decode_record(record, name_to_features):
    """Decodes a record to a TensorFlow example."""
    example = tf.parse_single_example(record, name_to_features)

    # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
    # So cast all int64 to int32.
    for name in list(example.keys()):
      t = example[name]
      if t.dtype == tf.int64:
        t = tf.cast(t, tf.int32)
      example[name] = t

    return example

  def input_fn(params, input_context=None):
    """The actual input function."""
    if FLAGS.use_tpu:
      batch_size = params["batch_size"]
    elif is_training:
      batch_size = FLAGS.train_batch_size
    elif FLAGS.do_eval:
      batch_size = FLAGS.eval_batch_size
    else:
      batch_size = FLAGS.predict_batch_size

    d = tf.data.TFRecordDataset(input_file)
    # Shard the dataset to difference devices
    if input_context is not None:
      tf.logging.info("Input pipeline id %d out of %d",
          input_context.input_pipeline_id, input_context.num_replicas_in_sync)
      d = d.shard(input_context.num_input_pipelines,
                  input_context.input_pipeline_id)

    # For training, we want a lot of parallel reading and shuffling.
    # For eval, we want no shuffling and parallel reading doesn't matter.
    if is_training:
      d = d.shuffle(buffer_size=FLAGS.shuffle_buffer)
      d = d.repeat()

    d = d.apply(
        tf.contrib.data.map_and_batch(
            lambda record: _decode_record(record, name_to_features),
            batch_size=batch_size,
            drop_remainder=drop_remainder))

    return d

  return input_fn

def file_based_input_fn_builder_pred(input_file, seq_length, is_training,
                                drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""


  name_to_features = {
      "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "input_mask": tf.FixedLenFeature([seq_length], tf.float32),
      "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "label_ids": tf.FixedLenFeature([31], tf.float32),
#       "label_ids": tf.FixedLenFeature([], tf.int64),
      "is_real_example": tf.FixedLenFeature([], tf.int64),
  }
  if FLAGS.is_regression:
    name_to_features["label_ids"] = tf.FixedLenFeature([31], tf.float32)
#     name_to_features["label_ids"] = tf.FixedLenFeature([], tf.float32)
  tf.logging.info("Input tfrecord file {}".format(input_file))

  def _decode_record(record, name_to_features):
    """Decodes a record to a TensorFlow example."""
    example = tf.parse_single_example(record, name_to_features)

    # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
    # So cast all int64 to int32.
    for name in list(example.keys()):
      t = example[name]
      if t.dtype == tf.int64:
        t = tf.cast(t, tf.int32)
      example[name] = t

    return example

  def input_fn(params, input_context=None):
    """The actual input function."""
    if FLAGS.use_tpu:
      batch_size = params["batch_size"]
    elif is_training:
      batch_size = FLAGS.train_batch_size
    elif FLAGS.do_eval:
      batch_size = FLAGS.eval_batch_size
    else:
      batch_size = FLAGS.predict_batch_size

    d = tf.data.TFRecordDataset(input_file)
    # Shard the dataset to difference devices
    if input_context is not None:
      tf.logging.info("Input pipeline id %d out of %d",
          input_context.input_pipeline_id, input_context.num_replicas_in_sync)
      d = d.shard(input_context.num_input_pipelines,
                  input_context.input_pipeline_id)

    # For training, we want a lot of parallel reading and shuffling.
    # For eval, we want no shuffling and parallel reading doesn't matter.
    if is_training:
      d = d.shuffle(buffer_size=FLAGS.shuffle_buffer)
      d = d.repeat()

    d = d.apply(
        tf.contrib.data.map_and_batch(
            lambda record: _decode_record(record, name_to_features),
            batch_size=batch_size,
            drop_remainder=drop_remainder))

    return d

  return input_fn


# In[ ]:


def file_based_convert_examples_to_features_dev(
    examples, label_list, max_seq_length, tokenize_fn, output_file,
    num_passes=1):
  """Convert a set of `InputExample`s to a TFRecord file."""

  # do not create duplicated records
  if tf.gfile.Exists(output_file) and not FLAGS.overwrite_data:
    tf.logging.info("Do not overwrite tfrecord {} exists.".format(output_file))
    return

  tf.logging.info("Create new tfrecord {}.".format(output_file))

  writer = tf.python_io.TFRecordWriter(output_file)

  if num_passes > 1:
    examples *= num_passes


  for (ex_index, example) in enumerate(examples):                 # iterating the data#############
    if ex_index % 10000 == 0:
      tf.logging.info("Writing example {} of {}".format(ex_index,
                                                        len(examples)))
### feaure: one data
# input_ids=input_ids,
# input_mask=input_mask,
# segment_ids=segment_ids,
# label_id=label_id)
    feature = convert_single_example_dev(ex_index, example, label_list,
                                     max_seq_length, tokenize_fn)

    def create_int_feature(values):
      f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
      return f

    def create_float_feature(values):
      f = tf.train.Feature(float_list=tf.train.FloatList(value=list(values)))
      return f

    features = collections.OrderedDict()
    features["input_ids"] = create_int_feature(feature.input_ids)
    features["input_mask"] = create_float_feature(feature.input_mask)
    features["segment_ids"] = create_int_feature(feature.segment_ids)
    

    if label_list is not None:        ###go into this requirement###
      features["label_ids"] = create_float_feature([float(i) for i in feature.label_id]) ## error ##
    else:
      features["label_ids"] = create_float_feature([float(feature.label_id)])
    features["is_real_example"] = create_int_feature(
        [int(feature.is_real_example)])


    tf_example = tf.train.Example(features=tf.train.Features(feature=features))
    writer.write(tf_example.SerializeToString())
  writer.close()


def file_based_convert_examples_to_features_pred(
    examples, label_list, max_seq_length, tokenize_fn, output_file,
    num_passes=1):
  """Convert a set of `InputExample`s to a TFRecord file."""

  # do not create duplicated records
  if tf.gfile.Exists(output_file) and not FLAGS.overwrite_data:
    tf.logging.info("Do not overwrite tfrecord {} exists.".format(output_file))
    return

  tf.logging.info("Create new tfrecord {}.".format(output_file))

  writer = tf.python_io.TFRecordWriter(output_file)

  if num_passes > 1:
    examples *= num_passes


  for (ex_index, example) in enumerate(examples):                 # iterating the data#############
    if ex_index % 10000 == 0:
      tf.logging.info("Writing example {} of {}".format(ex_index,
                                                        len(examples)))
### feaure: one data
# input_ids=input_ids,
# input_mask=input_mask,
# segment_ids=segment_ids,
# label_id=label_id)
    feature = convert_single_example_dev(ex_index, example, label_list,
                                     max_seq_length, tokenize_fn)

    def create_int_feature(values):
      f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
      return f

    def create_float_feature(values):
      f = tf.train.Feature(float_list=tf.train.FloatList(value=list(values)))
      return f

    features = collections.OrderedDict()
    features["input_ids"] = create_int_feature(feature.input_ids)
    features["input_mask"] = create_float_feature(feature.input_mask)
    features["segment_ids"] = create_int_feature(feature.segment_ids)
    

    if label_list is not None:        ###go into this requirement###
      features["label_ids"] = create_float_feature([float(i) for i in feature.label_id]) ## error ##
    else:
      features["label_ids"] = create_float_feature([float(feature.label_id)])
    features["is_real_example"] = create_int_feature(
        [int(feature.is_real_example)])


    tf_example = tf.train.Example(features=tf.train.Features(feature=features))
    writer.write(tf_example.SerializeToString())
  writer.close()


# In[ ]:


def main(_):
###################################################################
###################################################################
    score_reader = ScoreReader(
    '/u02/datasets/psytesting/scores_combined.csv',
    '/u02/texts/' ,count_words=False,
    system_idx=-1, fname_idx=-2,
    score_idx=list(range(31)),
    testname_idx=None)
    
    score_reader.normalize_scores()
    ## 365194 train scores, 25013 val scores, 16675 test scores
    ## 374377 train scores, 24959 val scores, 16638 test scores
    trainScores, valScores, testScores = score_reader.split_train_test(0.9,0.6)
    ########################### need to be the multiple times of batch size
    trainScores=trainScores[:360000]
    valScores=valScores[:24000]    
    testScores=testScores[:12000]
    ###########################
    filename_train=[]
    score_train=[]
    for i,data in enumerate(trainScores):
        filename_train.append(trainScores[i][0])
        score_train.append(trainScores[i][1])
        
    filename_dev=[]
    score_dev=[]
    for i,data in enumerate(valScores):
        filename_dev.append(valScores[i][0])
        score_dev.append(valScores[i][1])
        
    filename_test=[]
    score_test=[]
    for i,data in enumerate(testScores):
        filename_test.append(testScores[i][0])
        score_test.append(testScores[i][1])
#############################################################    
############## SETTING FOT THE FUNCTION #####################
#############################################################

    tf.logging.set_verbosity(tf.logging.INFO)

  #### Validate flags
    if FLAGS.save_steps is not None:
        FLAGS.iterations = min(FLAGS.iterations, FLAGS.save_steps)

    if FLAGS.do_predict:
        predict_dir = FLAGS.predict_dir
        if not tf.gfile.Exists(predict_dir):
          tf.gfile.MakeDirs(predict_dir)

    processor=GLUEProcessor(filename=filename_train, label=score_train, 
                            filename_dev=filename_dev,label_dev=score_dev,
                            filename_test=filename_test,label_test=score_test)


    if not FLAGS.do_train and not FLAGS.do_eval and not FLAGS.do_predict:
        raise ValueError(
        "At least one of `do_train`, `do_eval, `do_predict` or "
        "`do_submit` must be True.")

    if not tf.gfile.Exists(FLAGS.output_dir):
        tf.gfile.MakeDirs(FLAGS.output_dir)

    task_name = FLAGS.task_name.lower()


    
####################################################################
####################### tokenization ###############################
####################################################################

# changed
#     label_list = processor.get_labels() if not FLAGS.is_regression else None #  label list Nonetype
    label_list = processor.get_labels() if FLAGS.is_regression else None #  label list is not None 


#   def get_labels(self):
#     return [0.0]

    sp = spm.SentencePieceProcessor()
    sp.Load(FLAGS.spiece_model_file)
    def tokenize_fn(text):
        text=str(text[0])

        text = preprocess_text(text, lower=FLAGS.uncased)
        return encode_ids(sp, text)

    run_config = model_utils.configure_tpu(FLAGS)

    model_fn = get_model_fn(len(label_list) if label_list is not None else None)

    spm_basename = os.path.basename(FLAGS.spiece_model_file)

  # If TPU is not available, this will fall back to normal Estimator on CPU
  # or GPU.
    if FLAGS.use_tpu:
        estimator = tf.contrib.tpu.TPUEstimator(
            use_tpu=FLAGS.use_tpu,
            model_fn=model_fn,
            config=run_config,
            train_batch_size=FLAGS.train_batch_size,
            predict_batch_size=FLAGS.predict_batch_size,
            eval_batch_size=FLAGS.eval_batch_size)
    else:
        estimator = tf.estimator.Estimator(
            model_fn=model_fn,
            config=run_config)
    
######################################################################################
##############################TRAINING SETTING #######################################
######################################################################################

    if FLAGS.do_train:
        train_file_base = "{}.len-{}.train.tf_record".format(
            spm_basename, FLAGS.max_seq_length)
        train_file = os.path.join(FLAGS.output_dir, train_file_base)
        tf.logging.info("Use tfrecord file {}".format(train_file))

        train_examples = processor.get_train_examples(FLAGS.data_dir, 
                                                      filename_train,filename_dev ,filename_test,
                                                      score_train,score_dev,score_test) # get training data
        
        np.random.shuffle(train_examples)
        tf.logging.info("Num of train samples: {}".format(len(train_examples)))

        file_based_convert_examples_to_features(
            train_examples, label_list, FLAGS.max_seq_length, tokenize_fn,
            train_file, FLAGS.num_passes
        )

        train_input_fn = file_based_input_fn_builder(
            input_file=train_file,
            seq_length=FLAGS.max_seq_length,
            is_training=True,
            drop_remainder=True)

#         estimator.train(input_fn=train_input_fn,steps=1000, max_steps=FLAGS.train_steps)
        estimator.train(input_fn=train_input_fn, max_steps=FLAGS.train_steps)  
######################################################################################
##############################  VAL   SETTING ########################################
######################################################################################
    print('----------------------------------')        
    print('------start to evaluate ----------')
    print('----------------------------------')

    if FLAGS.do_eval:
        eval_examples = processor.get_dev_examples(FLAGS.data_dir, 
                                                      filename_train,filename_dev ,filename_test, 
                                                      score_train,score_dev,score_test)
    if FLAGS.do_predict:
        pred_examples = processor.get_test_examples(FLAGS.data_dir, 
                                                      filename_train,filename_dev ,filename_test,
                                                      score_train,score_dev,score_test)

        ## print(len(eval_examples)) - >25035
        ## print(eval_examples[1])   <__main__.InputExample object at 0x7fa0ff19ecc0>
        
# data_dir: Directory for input data."   default="/home/calvin/111/glue_data/STS-B/",
#         tf.logging.info("Num of eval/predicted samples: {}".format(len(eval_examples)))
    
    if FLAGS.do_eval:
    # TPU requires a fixed batch size for all batches, therefore the number
    # of examples must be a multiple of the batch size, or else examples
    # will get dropped. So we pad with fake examples which are ignored
    # later on. These do NOT count towards the metric (all tf.metrics
    # support a per-instance weight, and these get a weight of 0.0).
    #
    # Modified in XL: We also adopt the same mechanism for GPUs.
        while len(eval_examples) % FLAGS.eval_batch_size != 0:
          eval_examples.append(PaddingInputExample())

        eval_file_base = "{}.len-{}.{}.eval.tf_record".format(
            spm_basename, FLAGS.max_seq_length, FLAGS.eval_split)
        eval_file = os.path.join(FLAGS.output_dir, eval_file_base)

        file_based_convert_examples_to_features_dev(                             
            eval_examples, label_list, FLAGS.max_seq_length, tokenize_fn,
            eval_file)

        assert len(eval_examples) % FLAGS.eval_batch_size == 0
        eval_steps = int(len(eval_examples) // FLAGS.eval_batch_size)

        eval_input_fn = file_based_input_fn_builder_dev(
            input_file=eval_file,
            seq_length=FLAGS.max_seq_length,
            is_training=False,
            drop_remainder=True)

        # Filter out all checkpoints in the directory
        steps_and_files = []
        filenames = tf.gfile.ListDirectory(FLAGS.model_dir)

        for filename in filenames:
          if filename.endswith(".index"):
            ckpt_name = filename[:-6]
            cur_filename = join(FLAGS.model_dir, ckpt_name)
            global_step = int(cur_filename.split("-")[-1])
            tf.logging.info("Add {} to eval list.".format(cur_filename))
            steps_and_files.append([global_step, cur_filename])
        steps_and_files = sorted(steps_and_files, key=lambda x: x[0])

        # Decide whether to evaluate all ckpts
        if not FLAGS.eval_all_ckpt:
            aa=FLAGS.star_from_dev
            steps_and_files = steps_and_files[aa:]
            steps_and_files = steps_and_files[::10]

        eval_results = []

        for global_step, filename in sorted(steps_and_files, key=lambda x: x[0]):
          ret = estimator.evaluate(
              input_fn=eval_input_fn,
              steps=eval_steps,
              checkpoint_path=filename) #################### #### error #### 

          ret["step"] = global_step
          ret["path"] = filename

          eval_results.append(ret)

          tf.logging.info("=" * 80)
          log_str = "Eval result | "
          for key, val in sorted(ret.items(), key=lambda x: x[0]):
            log_str += "{} {} | ".format(key, val)
          tf.logging.info(log_str)

######################################################################################
############################## PREDICT SETTING #######################################
######################################################################################

    if FLAGS.do_predict:
        print('----------------------------------')        
        print('---start to compute Prediction----')
        print('----------------------------------')
        while len(pred_examples) % FLAGS.predict_batch_size != 0:
            pred_examples.append(PaddingInputExample())
            
        eval_file_base = "{}.len-{}.{}.predict.tf_record".format(
            spm_basename, FLAGS.max_seq_length, FLAGS.eval_split)
        eval_file = os.path.join(FLAGS.output_dir, eval_file_base)
        
        
        file_based_convert_examples_to_features_pred(
            pred_examples, label_list, FLAGS.max_seq_length, tokenize_fn,
            eval_file)
        assert len(pred_examples) % FLAGS.predict_batch_size == 0
        pred_steps = int(len(pred_examples))
#         pred_steps = int(len(pred_examples) // FLAGS.eval_batch_size)       
        eval_input_fn = file_based_input_fn_builder_pred(
            input_file=eval_file,
            seq_length=FLAGS.max_seq_length,
            is_training=False,
            drop_remainder=True)        
        
        pred_input_fn = file_based_input_fn_builder(
            input_file=eval_file,
            seq_length=FLAGS.max_seq_length,
            is_training=False,
            drop_remainder=True)
        
        steps_and_files = []
        filenames = tf.gfile.ListDirectory(FLAGS.model_dir)
        ##############

        for filename in filenames:
            if filename.endswith(".index"):
                ckpt_name = filename[:-6]
                cur_filename = join(FLAGS.model_dir, ckpt_name)
                global_step = int(cur_filename.split("-")[-1])
                tf.logging.info("Add {} to eval list.".format(cur_filename))
                steps_and_files.append([global_step, cur_filename])
        steps_and_files = sorted(steps_and_files, key=lambda x: x[0])

        # Decide whether to evaluate all ckpts
#         if not FLAGS.eval_all_ckpt:
        steps_and_files = steps_and_files[-1:]

        eval_results = []

        for global_step, filename in sorted(steps_and_files, key=lambda x: x[0]):
          ret = estimator.evaluate(
              input_fn=eval_input_fn,
              steps=pred_steps,
              checkpoint_path=filename) 

          ret["step"] = global_step
          ret["path"] = filename

          eval_results.append(ret)

          tf.logging.info("=" * 80)
          log_str = "Eval result | "
          for key, val in sorted(ret.items(), key=lambda x: x[0]):
            log_str += "{} {} | ".format(key, val)
          tf.logging.info(log_str)
        
        print('----------------------------------')        
        print('----start to compute Pearson------')
        print('----------------------------------')
    ################################################################################# Peason start
        
        predict_results = []
        label_results=[]
        pearson=[]
        with tf.gfile.Open(os.path.join(predict_dir, "{}.tsv".format(
            task_name)), "w") as fout:
            fout.write("index\tprediction\n")
            import copy
            import math
            
            
            for pred_cnt, result in enumerate(estimator.predict(
              input_fn=pred_input_fn,
              yield_single_examples=False,
              checkpoint_path=FLAGS.predict_ckpt)):

                if pred_cnt % 1000 == 0:
                    tf.logging.info("Predicting submission for example: {}".format(
                      pred_cnt))
##################

                logits = [float(x) for x in result["logits"].flat]
                list_pre=[]
    #             predict_results.append(logits)

                list_pre=list(result['labels']).copy()

                for i in range(len(result['labels'])):
                    if math.isnan(list_pre[i]) is False:
                        list_pre[i]=list(result['logits']).pop(0)
                        
                  
                ## need to add as many as batch size #####      
                pearson.append(list(zip(list(list_pre[:31]), list(result["labels"][:31]))))
                pearson.append(list(zip(list(list_pre[31:62]), list(result["labels"][31:62]))))      
                pearson.append(list(zip(list(list_pre[62:93]), list(result["labels"][62:93]))))
                pearson.append(list(zip(list(list_pre[93:124]), list(result["labels"][93:124])))) 
                pearson.append(list(zip(list(list_pre[124:155]), list(result["labels"][124:155]))))
                pearson.append(list(zip(list(list_pre[155:]), list(result["labels"][155:])))) 
                
            pearson=np.array(pearson) # dimension: [data count,31 ,2]
            
            pearson_res=[]
            from sklearn.metrics import mean_squared_error
            sk_mse_res=[]
            for i in range(31):
                
                register1=pearson[:,i,0]
#                 register1 = register1[np.logical_not(np.isnan(register1))]
                register2=pearson[:,i,1]
#                 register2 = register2[np.logical_not(np.isnan(register2))]
                register1 = register1[~np.isnan(register1)]
                register2 = register2[~np.isnan(register2)]    
                pearson_res.append(pd.Series(register1).corr(pd.Series(register2)))
                sk_mse_res.append(np.sqrt(mean_squared_error(register1,register2)))

#             pearson = list((zip(predict_results, label_results)))
            print('pearson ###################')
            print(list(pearson_res))
            print('sk_mse_res ################')                    
            print(list(sk_mse_res))
#             print('label_results   ###################')        
#             print(label_results)
##################
#                 logits = [float(x) for x in result["logits"].flat]
#                 #             list_pre=[]
#                 predict_results.append(logits)


        pearson_path = os.path.join(predict_dir, "{}.pearson.json".format(
            task_name))

        with tf.gfile.Open(pearson_path, "w") as fp:
            json.dump(pearson_res, fp, indent=4)
            
#         predict_json_path = os.path.join(predict_dir, "{}.label_results.json".format(
#             task_name))

#         with tf.gfile.Open(predict_json_path, "w") as fp:
#             json.dump(label_results, fp, indent=4)


if __name__ == "__main__":
  tf.app.run()


# In[ ]:


import json

with open('/home/calvin/xlnet/predict_result/sts-b.pearson.json') as json_file:
    data = json.load(json_file)
    print(data)


# In[ ]:


# with open('/home/calvin/xlnet/predict_result/sts-b.label_results.json') as json_file:
#     data = json.load(json_file)
#     print(data)



parsing 430158 scores...
achieving a safe and secure environment,achieving commercial outcomes,analyst_0,building capability for all,candidate_grade,caring,changing and improving,collaborating and partnering,customer_0,delivering at pace,delivering value for money,job_grade,leading and communicating,making effective decisions,managing a quality service,numerical,opq32 big 5 great 8 scales ocf_g8_f1_sten_new,opq32 big 5 great 8 scales ocf_g8_f2_sten_new,opq32 big 5 great 8 scales ocf_g8_f3_sten_new,opq32 big 5 great 8 scales ocf_g8_f4_sten_new,opq32 big 5 great 8 scales ocf_g8_f5_sten_new,opq32 big 5 great 8 scales ocf_g8_f6_sten_new,opq32 big 5 great 8 scales ocf_g8_f7_sten_new,opq32 big 5 great 8 scales ocf_g8_f8_sten_new,other competency/experience,persuading and influencing,professional competency,sat_0,seeing the big picture,showing drive and resilience,verbal,filename,system_id

system index: system_id

score index: ['achieving a safe and secure environment', 'achieving commercial

I0804 14:07:30.407771 139892428633856 model_utils.py:36] Single device mode.
I0804 14:07:31.410052 139892428633856 estimator.py:201] Using config: {'_tf_random_seed': None, '_is_chief': True, '_protocol': None, '_evaluation_master': '', '_master': '', '_task_id': 0, '_num_worker_replicas': 1, '_model_dir': 'exp', '_cluster': None, '_save_checkpoints_steps': 6000, '_tpu_config': TPUConfig(iterations_per_loop=400, num_shards=1, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_dims=None), '_train_distribute': None, '_service': None, '_save_checkpoints_secs': None, '_eval_distribute': None, '_keep_checkpoint_max': 0, '_experimental_distribute': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f39662d7940>, '_device_fn': None, '_task_type': 'worker', '_keep_checkpoint_every_n_hours': 10000, '_global_id_in_cluster': 0, '_log_step_count_steps': 100, '_num_ps_replicas': 0, '_sessi


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



I0804 14:07:49.425521 139892428633856 <ipython-input-2-e5508a133397>:976] Num of train samples: 360000
I0804 14:07:49.426978 139892428633856 <ipython-input-2-e5508a133397>:344] Do not overwrite tfrecord proc_data/sts-b/spiece.model.len-1024.train.tf_record exists.
I0804 14:07:49.427974 139892428633856 <ipython-input-2-e5508a133397>:411] Input tfrecord file proc_data/sts-b/spiece.model.len-1024.train.tf_record
W0804 14:07:49.447721 139892428633856 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
W0804 14:07:49.492398 139892428633856 deprecation.py:323] From <ipython-input-2-e5508a133397>:456: map_and_batch (from tensorflow.contrib.data.python.ops.batching) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.

I0804 14:07:55.340051 139892428633856 model_utils.py:91]   name = model/transformer/layer_2/rel_attn/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
I0804 14:07:55.340716 139892428633856 model_utils.py:91]   name = model/transformer/layer_2/ff/layer_1/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
I0804 14:07:55.341446 139892428633856 model_utils.py:91]   name = model/transformer/layer_2/ff/layer_1/bias:0, shape = (3072,), *INIT_FROM_CKPT*
I0804 14:07:55.342173 139892428633856 model_utils.py:91]   name = model/transformer/layer_2/ff/layer_2/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
I0804 14:07:55.342822 139892428633856 model_utils.py:91]   name = model/transformer/layer_2/ff/layer_2/bias:0, shape = (768,), *INIT_FROM_CKPT*
I0804 14:07:55.343461 139892428633856 model_utils.py:91]   name = model/transformer/layer_2/ff/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
I0804 14:07:55.344147 139892428633856 model_utils.py:91]   name = model/transformer/layer_2/ff/LayerNorm/gamma

...

I0808 12:28:40.705985 139892428633856 basic_session_run_hooks.py:247] loss = 0.69005114, step = 281100 (153.179 sec)
I0808 12:31:13.766173 139892428633856 basic_session_run_hooks.py:680] global_step/sec: 0.653328
I0808 12:31:13.769140 139892428633856 basic_session_run_hooks.py:247] loss = 0.9426013, step = 281200 (153.063 sec)
I0808 12:33:46.873149 139892428633856 basic_session_run_hooks.py:680] global_step/sec: 0.653138
I0808 12:33:46.876183 139892428633856 basic_session_run_hooks.py:247] loss = 0.578333, step = 281300 (153.107 sec)
I0808 12:36:20.048291 139892428633856 basic_session_run_hooks.py:680] global_step/sec: 0.652847
I0808 12:36:20.050914 139892428633856 basic_session_run_hooks.py:247] loss = 0.23742054, step = 281400 (153.175 sec)
I0808 12:38:53.084467 139892428633856 basic_session_run_hooks.py:680] global_step/sec: 0.65344
I0808 12:38:53.087214 139892428633856 basic_session_run_hooks.py:247] loss = 0.69033694, step = 281500 (153.036 sec)
I0808 12:41:26.167636 1398924286338

I0808 14:05:42.189899 139892428633856 basic_session_run_hooks.py:680] global_step/sec: 0.653154
I0808 14:05:42.192915 139892428633856 basic_session_run_hooks.py:247] loss = 0.3945621, step = 284900 (153.103 sec)
I0808 14:08:15.318714 139892428633856 basic_session_run_hooks.py:680] global_step/sec: 0.653045
I0808 14:08:15.321683 139892428633856 basic_session_run_hooks.py:247] loss = 0.7011837, step = 285000 (153.129 sec)
I0808 14:10:48.409833 139892428633856 basic_session_run_hooks.py:680] global_step/sec: 0.653206
I0808 14:10:48.412699 139892428633856 basic_session_run_hooks.py:247] loss = 0.92590666, step = 285100 (153.091 sec)
I0808 14:13:21.521231 139892428633856 basic_session_run_hooks.py:680] global_step/sec: 0.653119
I0808 14:13:21.523996 139892428633856 basic_session_run_hooks.py:247] loss = 0.62630063, step = 285200 (153.111 sec)
I0808 14:15:54.722975 139892428633856 basic_session_run_hooks.py:680] global_step/sec: 0.652734
I0808 14:15:54.726308 139892428633856 basic_session_ru

I0808 15:40:10.314529 139892428633856 basic_session_run_hooks.py:247] loss = 0.93400615, step = 288600 (153.070 sec)
I0808 15:42:43.463265 139892428633856 basic_session_run_hooks.py:680] global_step/sec: 0.65295
I0808 15:42:43.466010 139892428633856 basic_session_run_hooks.py:247] loss = 0.54572266, step = 288700 (153.151 sec)
I0808 15:45:16.474155 139892428633856 basic_session_run_hooks.py:680] global_step/sec: 0.653548
I0808 15:45:16.476638 139892428633856 basic_session_run_hooks.py:247] loss = 1.0368046, step = 288800 (153.011 sec)
I0808 15:47:49.626717 139892428633856 basic_session_run_hooks.py:680] global_step/sec: 0.652944
I0808 15:47:49.629554 139892428633856 basic_session_run_hooks.py:247] loss = 0.9821874, step = 288900 (153.153 sec)
I0808 15:50:22.781672 139892428633856 basic_session_run_hooks.py:680] global_step/sec: 0.652933
I0808 15:50:22.784513 139892428633856 basic_session_run_hooks.py:247] loss = 0.9663787, step = 289000 (153.155 sec)
I0808 15:52:55.892388 1398924286338

I0808 17:19:41.819325 139892428633856 basic_session_run_hooks.py:247] loss = 0.33970267, step = 292500 (153.091 sec)
I0808 17:22:14.967777 139892428633856 basic_session_run_hooks.py:680] global_step/sec: 0.652948
I0808 17:22:14.970265 139892428633856 basic_session_run_hooks.py:247] loss = 0.19211829, step = 292600 (153.151 sec)
I0808 17:24:48.076715 139892428633856 basic_session_run_hooks.py:680] global_step/sec: 0.65313
I0808 17:24:48.079325 139892428633856 basic_session_run_hooks.py:247] loss = 0.7956373, step = 292700 (153.109 sec)
I0808 17:27:21.183763 139892428633856 basic_session_run_hooks.py:680] global_step/sec: 0.653138
I0808 17:27:21.186918 139892428633856 basic_session_run_hooks.py:247] loss = 0.8994984, step = 292800 (153.108 sec)
I0808 17:29:54.254309 139892428633856 basic_session_run_hooks.py:680] global_step/sec: 0.653294
I0808 17:29:54.257443 139892428633856 basic_session_run_hooks.py:247] loss = 1.0184615, step = 292900 (153.071 sec)
I0808 17:32:27.408058 1398924286338

I0808 18:56:43.015164 139892428633856 basic_session_run_hooks.py:680] global_step/sec: 0.653496
I0808 18:56:43.018040 139892428633856 basic_session_run_hooks.py:247] loss = 0.76970834, step = 296300 (153.023 sec)
I0808 18:59:16.057844 139892428633856 basic_session_run_hooks.py:680] global_step/sec: 0.653413
I0808 18:59:16.060866 139892428633856 basic_session_run_hooks.py:247] loss = 0.51822144, step = 296400 (153.043 sec)
I0808 19:01:49.090399 139892428633856 basic_session_run_hooks.py:680] global_step/sec: 0.653456
I0808 19:01:49.093212 139892428633856 basic_session_run_hooks.py:247] loss = 0.72048724, step = 296500 (153.032 sec)
I0808 19:04:22.222403 139892428633856 basic_session_run_hooks.py:680] global_step/sec: 0.653031
I0808 19:04:22.225335 139892428633856 basic_session_run_hooks.py:247] loss = 0.86194944, step = 296600 (153.132 sec)
I0808 19:06:55.387885 139892428633856 basic_session_run_hooks.py:680] global_step/sec: 0.652889
I0808 19:06:55.390773 139892428633856 basic_session_

In [ ]:
#   file_based_convert_examples_to_features_dev(                             
#             eval_examples, label_list, FLAGS.max_seq_length, tokenize_fn,
#             eval_file)